In [ ]:
import os
import torch
from pathlib import Path
from tqdm import tqdm
from yolov5.utils.general import (check_dataset, check_img_size, non_max_suppression, scale_coords, xyxy2xywh)
from yolov5.utils.metrics import ap_per_class
from yolov5.utils.dataloaders import create_dataloader
import torch_pruning as tp

In [ ]:
device = "cuda:0"

In [ ]:
def prune(model, amount=0.3):
    import torch.nn.utils.prune as prune
    print('Pruning model... ', end='')
    for name, m in model.named_modules():
        if isinstance(m, torch.nn.Conv2d):
            prune.l1_unstructured(m, name='weight', amount=amount)  # prune
            prune.remove(m, 'weight')  # make permanent
    print(' %.3g global sparsity' % sparsity(model))
    return model
            
def sparsity(model):
    # Return global model sparsity
    a, b = 0, 0
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum()
    return b / a

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

def flops_and_params(model):
    example_inputs = torch.randn(1, 3, 640, 640).to(device)
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    flops = 2 * macs
    return flops, nparams

In [ ]:
def evaluate(model, data_yaml, device='cuda:0', img_size=640, batch_size=32, conf_thres=0.001, iou_thres=0.6):
    """
    YOLOv5 모델 테스트 함수 (2022년 초 버전 기반).
    
    Args:
        model (torch.nn.Module): PyTorch YOLOv5 모델 객체
        data_yaml (str or Path): 데이터셋 yaml 파일 경로
        device (str): 'cuda' 또는 'cpu'
        img_size (int): 입력 이미지 크기 (정사각형 기준)
        batch_size (int): 배치 크기
        conf_thres (float): Confidence threshold
        iou_thres (float): IoU threshold for NMS
    
    Returns:
        dict: 테스트 결과 메트릭 (Precision, Recall, mAP@0.5, mAP@0.5:0.95)
    """

    # 디바이스 설정
    device = torch.device(device)
    model.to(device).eval()

    # 데이터셋 확인
    data = check_dataset(data_yaml)
    names = data['names']

    # 데이터 로더 생성
    stride = int(model.stride.max())
    img_size = check_img_size(img_size, s=stride)
    _, dataloader, _ = create_dataloader(
        path=data['val'],
        imgsz=img_size,
        batch_size=batch_size,
        stride=stride,
        pad=0.5,
        rect=True,
        workers=4,
        prefix='val: '
    )

    # 테스트 루프
    stats, ap, ap_class = [], [], []
    for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc="Testing")):
        img = img.to(device, non_blocking=True).float() / 255.0  # 정규화
        targets = targets.to(device)

        # 추론
        with torch.no_grad():
            pred = model(img)
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)

        # 메트릭 계산
        for si, pred_i in enumerate(pred):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []
            correct = torch.zeros(pred_i.shape[0], nl, dtype=torch.bool, device=device)

            if len(pred_i):
                predn = pred_i.clone()
                scale_coords(img[si].shape[1:], predn[:, :4], shapes[si][0])  # 스케일 조정
                for ci, p in enumerate(predn):
                    ious = box_iou(p[None, :4], labels[:, 1:5]) > iou_thres
                    correct[ci, :] = ious

            stats.append((correct.cpu(), pred_i[:, 4].cpu(), pred_i[:, 5].cpu(), tcls))

    # AP 계산
    stats = [torch.cat(x, 0).numpy() for x in zip(*stats)]  # stats 압축
    p, r, ap, f1, ap_class = ap_per_class(*stats, plot=False)
    mp, mr, map50, map95 = p.mean(), r.mean(), ap[:, 0].mean(), ap.mean()  # 평균 계산

    # 결과 반환
    results = {
        'Precision': mp,
        'Recall': mr,
        'mAP@0.5': map50,
        'mAP@0.5:0.95': map95
    }
    print(results)
    return results

# Example Usage (adjust paths as needed):
# test_yolov5_v1(model, data_yaml='data/coco128.yaml', device='cuda')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pruning_ratios = np.linspace(0.0, 0.5, 6)  # 0%, 10%, ..., 50%
results = []

model = torch.hub.load('yolov5', 'custom', 'finetuned_weights/yolov5n_finetuned.pt', source='local', force_reload=True, device=device)

for ratio in pruning_ratios:
    print(ratio)
    pruned_model = prune(model, pruning_ratio=ratio)
    metrics = evaluate(model=model, data_yaml='./yolov5/data/glasses.yaml')
    results.append([ratio, metrics['Precision'], metrics['Recall'], metrics['mAP@0.5'], metrics['mAP@0.5:0.95']])
    print('{} complete'.format(ratio))

In [ ]:
# 데이터 추출
ratios = [r[0] for r in results]
precision_score = [r[1] for r in results]
recall_score = [r[2] for r in results]
map50_scores = [r[3] for r in results]
map50_95_scores = [r[4] for r in results]

# 그래프 그리기
plt.figure(figsize=(8, 6))
plt.plot(ratios, precision_score, marker='o', label='precision')
plt.plot(ratios, recall_score, marker='o', label='recall')
plt.plot(ratios, map50_scores, marker='o', label='mAP@0.5')
plt.plot(ratios, map50_95_scores, marker='s', label='mAP@0.5:0.95')

plt.title("YOLOv5 Pruning Ratio vs Performance")
plt.xlabel("Pruning Ratio")
plt.ylabel("Performance (mAP)")
plt.legend()
plt.grid(True)
plt.show()